In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select publisher_id, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_publisher_fact_v2 where date>='{}' and date<='{}' group by publisher_id".format(start,end)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start, end)
    return [Row(publisher_id=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("publisher_id",  LongType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by publisher_id desc")
print("=============Online============")
online_data_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-31"))
print("=============ADL============")

sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
result_df = spark.sql("""
     select publisher_key,
            sum(est_free_app_download) as sum_est_free_app_download,
            sum(est_paid_app_download) as sum_est_paid_app_download,
            sum(est_revenue) as sum_est_revenue
    from sample_data
    where device_code != 'ios-all'
    group by publisher_key
""")

print(sample_data_df.count())
online_data_df.exceptAll(result_df).show()
result_df.exceptAll(online_data_df).show()

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select date, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_publisher_fact_v2 where date>='{}' and date<='{}' group by date".format(start,end)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start, end)
    return [Row(date=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("date",  DateType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by date desc")
print("=============Online============")
online_data_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-31"))
print("=============ADL============")

sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
result_df = spark.sql("""
     select date,
            sum(est_free_app_download) as sum_est_free_app_download,
            sum(est_paid_app_download) as sum_est_paid_app_download,
            sum(est_revenue) as sum_est_revenue
    from sample_data
    where device_code != 'ios-all'
    group by date
""")

print(sample_data_df.count())
online_data_df.exceptAll(result_df).show()
result_df.exceptAll(online_data_df).show()

In [0]:

import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F
ios_feed = {1: "0,1,2,100,101,102"}
android_feed = {0: "0,1,2"}
def citus_row(start,end):
    def get_data_in_citus(start,end):
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = "select granularity, cast(sum(est_free_app_download) as bigint) as est_free_app_download, cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, cast(sum(est_revenue) as bigint) as est_revenue from store.store_est_publisher_fact_v2 where date>='{}' and date<='{}' group by granularity".format(start,end)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(start, end)
    return [Row(granularity=r[0], sum_est_free_app_download=r[1], sum_est_paid_app_download=r[2],
                sum_est_revenue=r[3]) for r in result]
def generate_citus_result(spark, citus_data):
    schema = StructType([
    StructField("granularity",  StringType(), True),
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True)])
    df_3 = spark.createDataFrame(citus_data, schema)
    df_3.createOrReplaceTempView("citus_data")
    return spark.sql("select * from citus_data order by granularity desc")
print("=============Online============")
online_data_df = generate_citus_result(spark,citus_row("2020-08-01","2020-08-31"))
print("=============ADL============")

sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
result_df = spark.sql("""
     select granularity,
            sum(est_free_app_download) as sum_est_free_app_download,
            sum(est_paid_app_download) as sum_est_paid_app_download,
            sum(est_revenue) as sum_est_revenue
    from sample_data
    where device_code != 'ios-all'
    group by granularity
""")

print(sample_data_df.count())
online_data_df.exceptAll(result_df).show()
result_df.exceptAll(online_data_df).show()

In [0]:

sample_data_df = spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.publisher-download-revenue.v1/fact/")
sample_data_df.createOrReplaceTempView("sample_data")
spark.sql("""
     select sum(est_paid_app_download) as paid,
            sum(est_free_app_download) as free,
            sum(est_revenue) as revenue,
            device_code
    from sample_data
    group by device_code
""").show()
print(sample_data_df.count())

In [0]:

spark.read.format("parquet").load("s3://b2c-prod-data-pipeline-unified-store-paid/_obselete/snowflake_v1/store.publisher-download-revenue.v1/fact/")

In [0]:
%%sh
